Title: Server_Client_TFFEMNIST_CIF_Group

Purpose: This is the group code intended to be updated by all

Contains: TFF-EMNIST database, CIF (Data Quality + SINR) function

Date of Creation: November 25th, 2020

### Import all Packages ###

In [39]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import collections
import nest_asyncio
%reload_ext tensorboard
nest_asyncio.apply()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

### Load in TFF - EMNIST Data Set ###

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

### Create a Server Class ###

In [46]:
class server:
    
    def __init__(self, numOfClients):
        self.numOfClients = numOfClients #Num of Client
        self.clientIds = list(range(0,self.numOfClients)) #Client List
        self.updateRoundNum = 0  # Update Round Number
        self.serverModel = self.createBaseModel() #Create a Keras Model for MNIST
        self.clientActive = [] #If client is available or not based on time - day/night
        self.clientCIF = []
        self.clientSelected = []
        self.maxClientsPerRound = 10
        self.updateFromClients=[]
        self.__serverTestData_X=[]
        self.__serverTestData_Y=[]
        self.__serverTestData = self.setServerTestDataTFF(emnist_test)
        self.predictionAcc=[]
        self.predictionLoss=[]
        
        
    def createBaseModel(self):
        #return tf.keras.models.Sequential([
        #    Dense(64, activation='relu',input_shape=(784,)),
        #    Dense(64, activation='relu'),
        #    Dense(10, activation='softmax'),])
        return tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(784,)),
            tf.keras.layers.Dense(10, kernel_initializer='zeros'),
            tf.keras.layers.Softmax()])
    
    def setServerTestData(self,Xtest,Ytest):  # NOT USED IN THIS CASE
        self.__serverTestData_X=Xtest
        self.__serverTestData_Y=Ytest
        
    def setServerTestDataTFF(self,serverTestData):  #Creates a test set for server
        BATCH_SIZE = 1000
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(10).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        serverData1 = serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[10])
        for i in range (2000,2500):
            serverData1 = serverData1.concatenate(serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[i]))
            
        serverData =tfds.as_numpy(preprocess(serverData1))
        return serverData
     
        
    def initialBroadcast(self):
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name, Using Client_List Dictionary
            #clientName = "client_"+str(self.clientIds[i])  #Create Client Name
            #print(clientName)
            clientName.setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            #eval(clientName).setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            
    def getClientActiveStatus(self):  #if the client is available for update or not randomly set in client
        print("--------------------------------------------\n","Round NO:",self.updateRoundNum)
        self.clientActive = []
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name
            #print(clientName)
            if(clientName.sendActiveStatus() == 1): #getActiveStatus() -> method of client class
                self.clientActive.append(i)
        print("Active Clients in Round :--: ", self.clientActive)
    
    
    def getClientCIF(self): # Get Client Importance Factor for active clients
        self.clientCIF=[]
        for i in self.clientActive:
            clientName = client_list["client_"+str(i)]
            c_cif = clientName.sendCIF()   #sendCIF() - method of client
            self.clientCIF.append(c_cif)
        print("Clients with Acceptable CIF: ", self.clientCIF)
        
    def getClientSelected(self): # Select The Top (N = maxClientsPerRound) with the highest CIF value
        self.clientSelected = []
        # Use non-class, local variables to leave class variables clean
        clientCIF, clientActive = ( list(t) for t in zip(*sorted(zip(self.clientCIF, self.clientActive)))) # Sort Active Clients and their CIF values, by CIF
        self.clientSelected  = clientActive[-self.maxClientsPerRound:] # Select the N=maxClientsPerRound of clients with highest CIF
        self.clientCIF = clientCIF[-self.maxClientsPerRound:] # Select the N=maxClientsPerRound of highest CIF
            
    
    def getModelUpdateFromClients(self):
        
        print("Active Clients: ",self.clientActive)
        print("CIF of Active Clients: ", self.clientCIF)
        print("Selected Clients: ", self.clientSelected)
        print("Training Clients with CIF > 10")
        self.updateRoundNum +=1
        self.updateFromClients=[]
        #self.dataPointsClients=[]
        
        for i in range (0,len(self.clientSelected)):
            if self.clientCIF[i] >= 10:  #Select Client only if CIF > some value 10 chosen for testing ONLY
                clientName = client_list["client_"+str(int(self.clientSelected[i]))]
                self.updateFromClients.append(clientName.sendClientUpdate()) #Get a list of updates from selected clients
            
        avgModelWeights = self.computeFedAvg(self.updateFromClients)
        #print("average model weights = \n",avgModelWeights)
        self.serverModel.set_weights(avgModelWeights)   
        
    def computeFedAvg(self,updates):  #Compute Federated Averaging from Available Clients
        totalDataPoints = 0
        scaleFactor=[]
        for i in range (0,len(updates)):
            totalDataPoints += updates[i][1]   #Sum the total Data Points on All Available Clients
            scaleFactor.append(updates[i][1])  #Store individual number datapoints for clients

        scaleFactor = np.array(scaleFactor)/totalDataPoints #Create the scale factor
       
        sumOfAvgWeights = []*len(updates[0][0]) 
        
        for j in range (0,len(updates[0][0])): #range of layers
            k=np.zeros_like(updates[0][0][j])
            #print("ShapeofK:",k.shape)
            for i in range(0,len(updates)): #range of clients
                k=k+updates[i][0][j]*scaleFactor[i]
                #print("ShapeofK:",k.shape)
            sumOfAvgWeights.append(k)
      
        return sumOfAvgWeights   
    
    def testServerModel(self):
        self.serverModel.compile(loss = 'sparse_categorical_crossentropy',
                                optimizer = tf.keras.optimizers.SGD (learning_rate=.01),
                                metrics=['accuracy'])
        testData = next(iter(self.__serverTestData))
        return self.serverModel.evaluate(testData['x'],
                                         testData['y'],)
        
        
    def updateAllClients(self):
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name
            clientName.setModelUpdateWeights(self.serverModel.get_weights()) 



### Create a Client Class ###

In [47]:
class client:
    
    def __init__(self,ID):
        self.id = ID
        self.__clientModel = tf.keras.models.Sequential()
        self.__clientModelWeights=[]
        self.activeStatus = 0
        self.__clientDataX=[] #Private to Client
        self.__clientDataY=[]
        self.clientCIF=0
        self.meanSINR = int(np.random.normal(15, 2.5, 1)) #Select random int from normal dist 
        self.roundSINR = 0
        self.propFair = 0
        self.epochs = 10
        self.lr = 0.01
        self.__clientData = self.__clientDataFromTFF(emnist_train)
        self.__currentData = next(iter(self.__clientData))
        self.dataQuality = 0
        
    def __clientDataFromTFF(self,tffDataset):
        NUM_EPOCHS = self.epochs
        BATCH_SIZE = 20
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        return tfds.as_numpy(preprocess(tffDataset.create_tf_dataset_for_client(tffDataset.client_ids[self.id])))
    
    
    #Methods Private to client
    def __setActiveStatus(self): 
        self.activeStatus = int(np.random.randint(0,2,1))  #01.01.21 Random Client Activation
        #self.activeStatus = 1
    
    def __setRoundSINR(self):
        self.roundSINR = np.round((np.random.normal(self.meanSINR, 1, 1)[0]), 2)
        #print("Round SNR for Client: ",self.id," is: ", self.roundSINR)
                               
    def __setPropFair(self):
        self.propFair = self.roundSINR / self.meanSINR
        
    def __setdataQuality(self):\
        
        ## Calculate for entropy - Start ##
        
#         for values in classification_probabilities:
#           for value in values:
#             score += value * np.log(value)
#         score = score * -1

        ## Calculate for entropy - End ##
        
        #self.dataQuality = 20 # Set 20 so that CIF value will always be passing
        
        # Next Client Data to be called  <<_ NOt written yet
        self.__currentData = next(iter(self.__clientData))
        predictions = self.__clientModel.predict(self.__currentData['x'])  # Get Class Predictions using Predict
        entropy = []*len(predictions)  # Entropy Will hold the entropy scores for all samples in the given round for this round
       
        for i in range (0,len(predictions)):
            sampleClassProb = predictions[i] #Take Class probability for every sample 
            score=0
            for value in sampleClassProb:  # Calculate the score - same as what jonny wrote in the pseudo code above
                #print('Value:',value)
                score += value*np.log2(value)
            score=score*(-1)
            #print('Score:',score)
            entropy.append(score)  # Append the Entropy for the given Sample
        #print('Entropy:',entropy)
        self.dataQuality = max(entropy) #Use the max (entropy) value as the data quality index
        #print("Data Quality for Client: ",self.id," is: ", self.dataQuality)
        self.dataQuality = (self.dataQuality-3)*100 #<<- All entropys are 3.something so normalized to get the dec values
        print("Data Quality for Client: ",self.id," is: ", self.dataQuality)
        
    def __setCIF(self):
        #self.clientCIF = int(np.random.randint(0,50,1))  # Random Number between 0 and 49 for check purpose only
        self.__setRoundSINR()
        self.__setPropFair() #This is currently unused
        self.__setdataQuality()
        self.clientCIF = self.roundSINR + self.dataQuality
        # self.clientCIF = (-1/self.roundSINR) + self.dataQuality #Formula for computing client CIF
        
    def __getTrainData(self):
        return self.__clientDataX, self.__clientDataY  # Just for Testing will be expanded to select meaningful data
    
    #Public Methods
    def sendActiveStatus(self):
        self.__setActiveStatus()
        return self.activeStatus
    
    def sendCIF(self):
        self.__setCIF()
        return self.clientCIF
    
    def setInitialModel(self,model):
        self.__clientModel=model
        print(self.__clientModel.summary)
         
    def setClientData(self,X,Y):
        self.__clientDataX = X
        self.__clientDataY = Y
   

    def sendClientUpdate(self):
        print("->",self.id,end=" ")
        self.__clientModel.compile(loss = 'sparse_categorical_crossentropy',
                                optimizer = tf.keras.optimizers.SGD (learning_rate=self.lr),
                                metrics=['accuracy'])
        #print(self.__clientDataX.shape)
        #print(self.__clientDataY.shape)
        if len(self.__clientDataX) > 0:
            history = self.__clientModel.fit(self.__clientDataX,
                                   tf.keras.utils.to_categorical(self.__clientDataY),
                                   epochs=self.epochs,
                                   validation_split=0.2,
                                   verbose=0,
                                    shuffle=True)

            updatedWeights = self.__clientModel.get_weights()
        
        else:
            #currentData = next(iter(self.__clientData))  # <-- Shifted to the setDataQuality Function
            history = self.__clientModel.fit(self.__currentData['x'],
                                   self.__currentData['y'],
                                   epochs=self.epochs,
                                   verbose=0,shuffle=True)
            
            
            
            
            updatedWeights = self.__clientModel.get_weights()
       
        
        numOfDataPoints = len(self.__currentData['x'])   #<-- Set the correct type to get the number of datapoints in the current update round

        return (updatedWeights,numOfDataPoints)
    
    #def logTraningHistory(self,history,roundNumber):
    #    loss_history = history.history["loss"]
    #    acc_history = history.history["accuracy"]
    #    numpy_loss_history = numpy.array(loss_history)
    
    def setModelUpdateWeights(self,modelUpdateWeights):
        self.__clientModel.set_weights(modelUpdateWeights)
        #self.__clientModel.
        
    def plotClientData(self):
        #print("Y_Data",self.__clientDataY)
        id0=[]
        id1=[]
        id2=[]
        id3=[]
        id4=[]
        id5=[]
        id6=[]
        id7=[]
        id8=[]
        id9=[]
        for i in range (0,len(self.__clientData['x'])):
            for j in range (0,10):
                if j==self.__clientData['y']:
                    eval('id'+str(j)).append(i)
        xAxis=['0','1','2','3','4','5','6','7','8','9']
        yAxis=[len(id0),len(id1),len(id2),len(id3),len(id4),len(id5),len(id6),len(id7),len(id8),len(id9)]
        plt.bar(xAxis,yAxis)
        title="Data for Client"+str(self.id)
        plt.title(title)
        plt.show()

### Initialize Clients ###

In [48]:
client_list = {}

# Set the number of clients
num_clients = 100
for i in range(num_clients):
    name = "client_" + str(i)
    client_list[name] = client(i)

### Initialize Server ###

In [49]:
# Initialize server with number of clients
serverFA = server(num_clients)

### Initial Broadcast from Server to Client ###

In [50]:
serverFA.initialBroadcast()

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001CC98C67DC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

In [51]:
trainingName = "EntropyMaxC10_C100R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getClientActiveStatus()
        serverFA.getClientCIF()
        serverFA.getClientSelected()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


--------------------------------------------
 Round NO: 0
Active Clients in Round :--:  [0, 2, 4, 5, 6, 9, 10, 11, 13, 14, 21, 23, 29, 32, 34, 35, 41, 42, 44, 47, 51, 54, 57, 58, 61, 63, 64, 71, 74, 78, 79, 81, 83, 84, 91, 92, 93, 95, 97]
Round SNR for Client:  0  is:  15.29
Data Quality for Client:  0  is:  32.192808389663696
Round SNR for Client:  2  is:  14.16
Data Quality for Client:  2  is:  32.192808389663696
Round SNR for Client:  4  is:  14.27
Data Quality for Client:  4  is:  32.192808389663696
Round SNR for Client:  5  is:  12.7
Data Quality for Client:  5  is:  32.192808389663696
Round SNR for Client:  6  is:  19.56
Data Quality for Client:  6  is:  32.192808389663696
Round SNR for Client:  9  is:  17.01
Data Quality for Client:  9  is:  32.192808389663696
Round SNR for Client:  10  is:  19.62
Data Quality for Client:  10  is:  32.192808389663696
Round SNR for Client:  11  is:  15.11
Data Quality for Client:  11  is:  32.192808389663696
Round SNR for Client:  13  is:  10.12


Data Quality for Client:  88  is:  24.665048718452454
Round SNR for Client:  89  is:  12.37
Data Quality for Client:  89  is:  24.30710941553116
Round SNR for Client:  90  is:  16.35
Data Quality for Client:  90  is:  24.12996143102646
Round SNR for Client:  93  is:  12.52
Data Quality for Client:  93  is:  23.282083868980408
Round SNR for Client:  94  is:  13.71
Data Quality for Client:  94  is:  24.47401136159897
Round SNR for Client:  95  is:  13.42
Data Quality for Client:  95  is:  24.829328060150146
Round SNR for Client:  97  is:  11.69
Data Quality for Client:  97  is:  23.49495440721512
Round SNR for Client:  98  is:  13.6
Data Quality for Client:  98  is:  25.01130849123001
Round SNR for Client:  99  is:  13.85
Data Quality for Client:  99  is:  23.986171185970306
Clients with Acceptable CIF:  [33.3688072347641, 38.54441264152527, 40.411794781684875, 36.97612546205521, 41.555947058200836, 40.48356015205383, 37.190479419231416, 39.697181487083434, 39.02943893909455, 37.24423829

Data Quality for Client:  8  is:  29.911065101623535
Round SNR for Client:  11  is:  16.28
Data Quality for Client:  11  is:  30.481907725334167
Round SNR for Client:  13  is:  10.45
Data Quality for Client:  13  is:  29.719537496566772
Round SNR for Client:  15  is:  14.2
Data Quality for Client:  15  is:  30.212566256523132
Round SNR for Client:  17  is:  14.3
Data Quality for Client:  17  is:  30.22412359714508
Round SNR for Client:  18  is:  14.61
Data Quality for Client:  18  is:  30.09103536605835
Round SNR for Client:  21  is:  12.29
Data Quality for Client:  21  is:  30.77818751335144
Round SNR for Client:  22  is:  13.61
Data Quality for Client:  22  is:  30.20479679107666
Round SNR for Client:  23  is:  15.32
Data Quality for Client:  23  is:  30.32475709915161
Round SNR for Client:  29  is:  15.05
Data Quality for Client:  29  is:  28.9666086435318
Round SNR for Client:  31  is:  16.35
Data Quality for Client:  31  is:  30.18273115158081
Round SNR for Client:  32  is:  9.75


Data Quality for Client:  50  is:  28.24091613292694
Round SNR for Client:  51  is:  15.56
Data Quality for Client:  51  is:  29.692840576171875
Round SNR for Client:  54  is:  11.43
Data Quality for Client:  54  is:  27.757859230041504
Round SNR for Client:  58  is:  10.34
Data Quality for Client:  58  is:  28.921693563461304
Round SNR for Client:  59  is:  16.62
Data Quality for Client:  59  is:  28.5793736577034
Round SNR for Client:  61  is:  12.54
Data Quality for Client:  61  is:  27.502518892288208
Round SNR for Client:  62  is:  15.26
Data Quality for Client:  62  is:  30.254948139190674
Round SNR for Client:  63  is:  16.75
Data Quality for Client:  63  is:  28.923073410987854
Round SNR for Client:  70  is:  17.99
Data Quality for Client:  70  is:  28.282880783081055
Round SNR for Client:  71  is:  14.02
Data Quality for Client:  71  is:  28.776565194129944
Round SNR for Client:  72  is:  16.19
Data Quality for Client:  72  is:  29.05868887901306
Round SNR for Client:  75  is:

32/32 [==============================] - 0s 966us/step - loss: 1.9621 - accuracy: 0.3100
--------------------------------------------
 Round NO: 6
Active Clients in Round :--:  [2, 4, 7, 8, 9, 11, 14, 17, 19, 20, 26, 27, 29, 30, 31, 32, 35, 37, 43, 44, 49, 50, 51, 56, 57, 58, 59, 60, 61, 62, 65, 67, 69, 70, 71, 72, 73, 78, 80, 83, 84, 86, 87, 89, 91, 93, 94, 98]
Round SNR for Client:  2  is:  12.52
Data Quality for Client:  2  is:  27.44249701499939
Round SNR for Client:  4  is:  16.16
Data Quality for Client:  4  is:  28.341826796531677
Round SNR for Client:  7  is:  13.29
Data Quality for Client:  7  is:  25.64249485731125
Round SNR for Client:  8  is:  11.79
Data Quality for Client:  8  is:  27.588991820812225
Round SNR for Client:  9  is:  14.93
Data Quality for Client:  9  is:  26.711753010749817
Round SNR for Client:  11  is:  12.86
Data Quality for Client:  11  is:  26.63111537694931
Round SNR for Client:  14  is:  16.7
Data Quality for Client:  14  is:  27.097876369953156
Round

...

32/32 [==============================] - 0s 966us/step - loss: 1.3258 - accuracy: 0.5980
--------------------------------------------
 Round NO: 45
Active Clients in Round :--:  [0, 2, 3, 4, 5, 9, 10, 11, 12, 13, 15, 24, 27, 29, 31, 32, 33, 34, 35, 37, 38, 39, 43, 44, 46, 47, 48, 49, 50, 52, 56, 57, 60, 61, 62, 63, 65, 66, 67, 69, 70, 72, 73, 77, 78, 79, 81, 82, 83, 84, 88, 90, 92, 96, 97, 98]
Round SNR for Client:  0  is:  13.8
Data Quality for Client:  0  is:  2.6087164878845215
Round SNR for Client:  2  is:  14.78
Data Quality for Client:  2  is:  -28.53916846215725
Round SNR for Client:  3  is:  17.99
Data Quality for Client:  3  is:  8.473823964595795
Round SNR for Client:  4  is:  15.6
Data Quality for Client:  4  is:  -34.07253921031952
Round SNR for Client:  5  is:  13.92
Data Quality for Client:  5  is:  -38.77583313733339
Round SNR for Client:  9  is:  15.11
Data Quality for Client:  9  is:  -13.261996116489172
Round SNR for Client:  10  is:  18.67
Data Quality for Client:  1

Data Quality for Client:  23  is:  -8.137047290802002
Round SNR for Client:  24  is:  15.81
Data Quality for Client:  24  is:  8.198974281549454
Round SNR for Client:  25  is:  16.26
Data Quality for Client:  25  is:  -23.277434706687927
Round SNR for Client:  27  is:  21.78
Data Quality for Client:  27  is:  -10.71215532720089
Round SNR for Client:  28  is:  13.19
Data Quality for Client:  28  is:  -7.033656537532806
Round SNR for Client:  29  is:  15.91
Data Quality for Client:  29  is:  0.44654905796051025
Round SNR for Client:  31  is:  15.32
Data Quality for Client:  31  is:  -15.580491721630096
Round SNR for Client:  33  is:  15.51
Data Quality for Client:  33  is:  4.827502369880676
Round SNR for Client:  34  is:  16.89
Data Quality for Client:  34  is:  -16.710981726646423
Round SNR for Client:  37  is:  11.99
Data Quality for Client:  37  is:  -8.267293125391006
Round SNR for Client:  38  is:  14.87
Data Quality for Client:  38  is:  8.7393157184124
Round SNR for Client:  39  

Data Quality for Client:  48  is:  -22.58557565510273
Round SNR for Client:  49  is:  18.07
Data Quality for Client:  49  is:  -21.956881694495678
Round SNR for Client:  50  is:  16.75
Data Quality for Client:  50  is:  -15.827064216136932
Round SNR for Client:  52  is:  18.49
Data Quality for Client:  52  is:  8.081118762493134
Round SNR for Client:  55  is:  14.04
Data Quality for Client:  55  is:  -43.558745831251144
Round SNR for Client:  59  is:  15.83
Data Quality for Client:  59  is:  -41.631538048386574
Round SNR for Client:  60  is:  13.28
Data Quality for Client:  60  is:  4.815161228179932
Round SNR for Client:  62  is:  13.93
Data Quality for Client:  62  is:  -39.20927345752716
Round SNR for Client:  68  is:  18.14
Data Quality for Client:  68  is:  -4.846878349781036
Round SNR for Client:  70  is:  15.55
Data Quality for Client:  70  is:  10.350361466407776
Round SNR for Client:  72  is:  15.93
Data Quality for Client:  72  is:  -3.5119518637657166
Round SNR for Client:  

Data Quality for Client:  79  is:  -16.21430478990078
Round SNR for Client:  80  is:  14.96
Data Quality for Client:  80  is:  -13.530746474862099
Round SNR for Client:  83  is:  16.73
Data Quality for Client:  83  is:  -0.12024715542793274
Round SNR for Client:  85  is:  14.06
Data Quality for Client:  85  is:  2.8827890753746033
Round SNR for Client:  86  is:  16.57
Data Quality for Client:  86  is:  2.126975357532501
Round SNR for Client:  87  is:  15.57
Data Quality for Client:  87  is:  -31.959541514515877
Round SNR for Client:  88  is:  14.75
Data Quality for Client:  88  is:  -4.341878741979599
Round SNR for Client:  91  is:  9.88
Data Quality for Client:  91  is:  -22.661463916301727
Round SNR for Client:  95  is:  14.47
Data Quality for Client:  95  is:  -35.786159336566925
Round SNR for Client:  96  is:  18.7
Data Quality for Client:  96  is:  -26.296470127999783
Round SNR for Client:  98  is:  13.23
Data Quality for Client:  98  is:  2.605537325143814
Round SNR for Client:  

32/32 [==============================] - 0s 966us/step - loss: 1.3573 - accuracy: 0.5600


In [1]:
plt.plot(serverTestMetrics)


NameError: name 'plt' is not defined

In [53]:
%tensorboard --logdir logs/ 

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9024.

In [ ]:
testAcc=[]*numberOfTraingRounds
testLoss=[]*numberOfTraingRounds
for i in range(0,numberOfTraingRounds):
    testAcc.append(trainingMetrics[i][1])
    testLoss.append(trainingMetrics[i][0])
k = np.array(testLoss)

In [ ]:
k.

In [ ]:
np.savetxt("loss_history.txt", np.array(testLoss))

In [ ]:
meanSINR = int(np.random.normal(15, 2.5, 1))
print(meanSINR)

In [ ]:
np.round((np.random.normal(meanSINR, 1, 1)[0]), 2)

In [ ]:
raise SystemExit("Stop right there!")


In [ ]:
# calculate the information for a dice roll
from math import log2
# probability of the event
p = 1.0 / 6.0
# calculate information for event
h = -log2(p)
# print the result
print('p(x)=%.3f, information: %.3f bits' % (p, h))

In [ ]:
h

In [ ]:
probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
from matplotlib import pyplot
info = [-log2(p) for p in probs]
# plot probability vs information
pyplot.plot(probs, info, marker='.')
pyplot.title('Probability vs Information')
pyplot.xlabel('Probability')
pyplot.ylabel('Information')
pyplot.show()